In [1]:
import pandas as pd
import numpy as np

import itertools
import os

In [2]:
# Ignore 'trade_update_date', 'run_date', 'vessel_name', 'secondary_notify_party_1','container_number'
usecols = ['identifier','shipper_party_name', 'shipper_address','harmonized_number']

In [3]:
dtype = {'identifier':str,'shipper_party_name':str,'shipper_address':str,'harmonized_number':str}

In [4]:
#Enigma = pd.read_csv('../Data/Dataset/BofL6country.zip',usecols=usecols,dtype=dtype,compression='zip')

In [5]:
Enigma = pd.read_csv('../Data/Dataset/BillofLadingSummary-2018.zip',usecols=usecols,dtype=dtype,compression='zip')
#Enigma = Enigma.append(pd.read_csv('../Data/Dataset/BillofLadingSummary-2017.zip',usecols=usecols,dtype=dtype,compression='zip'),ignore_index=True)

In [6]:
Enigma = Enigma.dropna(subset=['shipper_party_name'])
Enigma = Enigma.reset_index(drop=True)
# Clean Enigma and Camaras and turn them into series
replace_char = ",.+=_-><\'\":;()!?~/\\@#$%^&*~`[]{}"
replace_dict = {key:value for (key,value) in zip(replace_char,itertools.repeat(''))}
Enigma['cl_shipper_party_name'] = Enigma['shipper_party_name'].str.translate(str.maketrans(replace_dict))
Enigma = Enigma.drop_duplicates(subset=['cl_shipper_party_name'])
Enigma = Enigma.reset_index(drop=True)

In [10]:
def create_slurm_job(shipper_size,n_steps,year):
    import math
    increment = math.floor(shipper_size/n_steps)
    start_from = 1
    for i in range(n_steps):
        if i == n_steps-1:
            print('srun --nodes=1 --ntasks=1 --exclusive python ~/FEIII-SHIP/shipper_matching/Enigma_Enigma_matching.py {} {} {}_{}.pkl.zip &'.format(start_from,shipper_size-1,year,i))
        else:
            print('srun --nodes=1 --ntasks=1 --exclusive python ~/FEIII-SHIP/shipper_matching/Enigma_Enigma_matching.py {} {} {}_{}.pkl.zip &'.format(start_from,start_from+increment,year,i))
        start_from = start_from + increment + 1

In [11]:
create_slurm_job(len(Enigma),10,2018)

srun --nodes=1 --ntasks=1 --exclusive python ~/FEIII-SHIP/shipper_matching/Enigma_Enigma_matching.py 1 50974 2018_0.pkl.zip &
srun --nodes=1 --ntasks=1 --exclusive python ~/FEIII-SHIP/shipper_matching/Enigma_Enigma_matching.py 50975 101948 2018_1.pkl.zip &
srun --nodes=1 --ntasks=1 --exclusive python ~/FEIII-SHIP/shipper_matching/Enigma_Enigma_matching.py 101949 152922 2018_2.pkl.zip &
srun --nodes=1 --ntasks=1 --exclusive python ~/FEIII-SHIP/shipper_matching/Enigma_Enigma_matching.py 152923 203896 2018_3.pkl.zip &
srun --nodes=1 --ntasks=1 --exclusive python ~/FEIII-SHIP/shipper_matching/Enigma_Enigma_matching.py 203897 254870 2018_4.pkl.zip &
srun --nodes=1 --ntasks=1 --exclusive python ~/FEIII-SHIP/shipper_matching/Enigma_Enigma_matching.py 254871 305844 2018_5.pkl.zip &
srun --nodes=1 --ntasks=1 --exclusive python ~/FEIII-SHIP/shipper_matching/Enigma_Enigma_matching.py 305845 356818 2018_6.pkl.zip &
srun --nodes=1 --ntasks=1 --exclusive python ~/FEIII-SHIP/shipper_matching/Enigma_E

### Result

In [2]:
path = 'match_by_levenshtein/'
data = pd.concat(pd.read_pickle(path + f,compression='zip') for f in os.listdir(path))

In [12]:
cleaned_data = data[((data['name_score']>0.9) & (data['address_score']>0.6)) | (data['address_score']>0.8)]

In [13]:
cleaned_data.shape

(369364, 8)

In [4]:
data.shape

(17702456, 8)

In [15]:
cleaned_data.to_csv('Enigma_Enigma_2018.csv',index=False)

In [ ]:
test['identifier_right'].iloc[2]

In [ ]:
data.reset_index(drop=True).to_csv('Enigma_Enigma_6countries.csv',index=False)

In [ ]:
data = data.reset_index(drop=True)

In [ ]:
data['name_score'].mean()

In [ ]:
data['address_score'].mean()

In [ ]:
data[data['name_score'] >= 0.85]['address_score'].mean()

In [ ]:
data[data['name_score'] >= 0.90]['address_score'].mean()

In [ ]:
data[data['name_score'] >= 0.95]['address_score'].mean()

In [ ]:
data.shape

In [ ]:
data[data['address_score']>=0.6].shape

In [ ]:
data[data['address_score']>=0.6]['name_score'].mean()

In [ ]:
scatter_plot = data.plot.scatter('name_score','address_score',s=0.5)
fig = scatter_plot.get_figure()
fig.savefig("name_address.png",dpi=200)

In [ ]:
data[(data['name_score'] >= 0.95) & (data['address_score']<=0.5)]